# 07 Experiment Tracking Class

In [19]:
from helper_functions import plot_predictions, plot_decision_boundary, accuracy_fn
from going_modular import engine, data_loaders
from going_modular.engine import train_step, test_step
import mlxtend
from mlxtend.plotting import plot_confusion_matrix
import numpy as np
import os
import pandas as pd
from pathlib import Path
import random
import requests
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
from sklearn.datasets import make_circles
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
from torchinfo import summary
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from torchmetrics import Accuracy, ConfusionMatrix
import torchvision
from torchvision import datasets

from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Dataset
from timeit import default_timer as timer
from tqdm.auto import tqdm
from typing import Tuple, Dict, List
writer = SummaryWriter()
import zipfile



In [2]:
# see torch and torch vision plus
print (torch.__version__)
print (torchvision.__version__)

2.3.0
0.18.0


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def set_seeds (seed: int=42):
    """_summary_

    Args:
        seed (int, optional): _description_. Defaults to 42.
    """ 
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)   

In [4]:
# do helder function
def download_data (source: str,
                   destination: str,
                   data_dir: str = '../data',
                   remove_source: bool = True) -> Path:
    """_summary_

    Args:
        soure (str): _description_
        destination (str): _description_
        remove_source (bool, optional): _description_. Defaults to True.

    Returns:
        Path: _description_
    """    

    data_path = Path (data_dir)
    image_path = data_path / destination
    if image_path.is_dir():
        print (f' {image_path} directory exists skipping download')
    else:
        print (f'[INFO] creating {image_path}')
        image_path.mkdir(parents = True, exist_ok = True)
        target_file = Path(source).name
        with open(data_path/ target_file, 'wb') as f:
            request = requests.get(source)
            print (f'[INFO] downloading {target_file} from {source}')
            f.write (request.content)
        with zipfile.ZipFile(data_path/target_file, 'r') as zip_ref:
            print (f'[INFO] Unsipping {target_file}')
            zip_ref.extractall(image_path)

        if remove_source:
            os.remove (data_path/target_file)
   
    return image_path


In [5]:
# data_path = Path('../data')
image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi_1")

 ..\data\pizza_steak_sushi_1 directory exists skipping download


In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [7]:
# manual data loader
image_path = Path('../data/pizza_steak_sushi_1')
train_dir = image_path /'train'
test_dir = image_path/'test'

normalize = transforms.Normalize(mean =[0.485, 0.456, 0.406],
                                 std = [0.229, 0.224, 0.225])
manual_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        normalize
])

train_dataloader, test_dataloader, class_names = data_loaders.create_dataloaders(
    train_dir = train_dir,
    test_dir = test_dir,
    transform = manual_transforms,
    batch_size = 32
)

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x1cabcb9f610>,
 ['pizza', 'steak', 'sushi'])

In [8]:
# auto data laoder
image_path = Path('../data/pizza_steak_sushi_1')
train_dir = image_path /'train'
test_dir = image_path/'test'

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
automatic_transforms = weights.transforms()
print (f'Auto transforms: {automatic_transforms}')

train_dataloader, test_dataloader, class_names= data_loaders.create_dataloaders(
    train_dir=train_dir,
    test_dir = test_dir,
    transform=automatic_transforms,
    batch_size = 32
)

train_dataloader, test_dataloader, class_names

Auto transforms: ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


(<torch.utils.data.dataloader.DataLoader at 0x1cabcbb3e50>,
 ['pizza', 'steak', 'sushi'])

## get pretrained model.



In [9]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model_B0 = torchvision.models.efficientnet_b0(weights = weights).to(device)
model_B0

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [10]:
for param in model_B0.features.parameters():
    param.requires_grad = False
    
set_seeds()

model_B0.classifier= torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280,
              out_features = len(class_names),
              bias = True).to(device)
)

In [11]:
from torchinfo import summary
summary(model_B0,
        input_size = (32, 3, 224, 224),
        verbose = 0,
        col_names = ['input_size', 'output_size', 'num_params','trainable' ],
        col_width = 20,
        row_settings = ['var_names'])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

## Train model

In [12]:
# start training
loss_fn = nn.CrossEntropyLoss()
optimizer    = torch.optim.Adam(model_B0.parameters(), lr=0.001)

In [16]:
writer = SummaryWriter()

### create  writer



In [69]:
def create_writer(experiment_name: str,
                model_name:str,
                extra: str=None) -> torch.utils.tensorboard.writer.SummaryWriter():

    """_summary_
    """    
    from datetime import datetime
    import os

    timestamp = datetime.now().strftime("%Y-%m-%d")

    if extra:
        log_dir = os.path.join('runs', timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.john ('runs', timestamp, experiment_name, model_name)

    print (f'[INFO] Created SummaryWriter, saving to: {log_dir}..')
    
    return SummaryWriter(log_dir = log_dir)

In [26]:
# import torch.utils


# def train (model: torch.nn.Module,
#            train_dataloader: torch.utils.data.DataLoader,
#            test_dataloader: torch.utils.data.DataLoader,
#            optimizer: torch.optim.Optimizer,
#            loss_fn: torch.nn.Module,
#            epochs: int,
#            device: torch.device) -> Dict[str, List]:
#     """_summary_

#     Args:
#         model (torch.nn.Module): _description_
#         train_dataloader (torch.utils.data.DataLoader): _description_
#         test_dataloader (torch.utils.data.DataLoader): _description_
#         optimizer (torch.optim.Optimizer): _description_
#         loss_fn (torch.nn.Module): _description_
#         epochs (int): _description_
#         device (torch.device): _description_

#     Returns:
#         Dict[str, List]: _description_
#     """   

#     results = {'train_loss':[],
#               'train_acc': [],
#               'test_loss': [],
#               'test_acc': []} 
    
#     for epoch in tqdm(range(epochs)):
#         train_loss, train_acc = train_step(model=model,
#                                            dataloader= train_dataloader,
#                                            loss_fn = loss_fn,
#                                            optimizer=optimizer,
#                                            device = device)
        
#         test_loss, test_acc = test_step(model=model,
#                                         dataloader = test_dataloader,
#                                         loss_fn=loss_fn,
#                                         device=device)
        
#         print(
#             f'Epoch: {epoch + 1} |'
#             f'train_loss: {train_loss:.4f} |'
#             f'train_acc: {train_acc:.4f} |'
#             f'test_lossL {test_loss:.4f} | '
#             f'test_acc: {test_acc:.4f}'
#         )

#         results['test_loss'].append(train_loss)
#         results['train_acc'].append(train_acc)
#         results['test_loss'].append(test_loss)
#         results['test_acc'].append(test_acc)

#         writer.add_scalars (main_tag = 'Accuracy',
#                             tag_scalar_dict = {'train_acc': train_acc,
#                                                'test_acc': test_acc},
#                                                global_step = epoch)
        
#         writer.add_graph (model= model,
#                           input_to_model=torch.randn(32, 3, 224, 224).to(device))
        
#     writer.close()

#     return results


In [84]:
import torch.utils
import torch.utils.tensorboard


def train (model: torch.nn.Module,
           train_dataloader: torch.utils.data.DataLoader,
           test_dataloader: torch.utils.data.DataLoader,
           optimizer: torch.optim.Optimizer,
           loss_fn: torch.nn.Module,
           epochs: int,
           device: torch.device,
           writer: torch.utils.tensorboard.writer.SummaryWriter) -> Dict[str, List]:
    """_summary_

    Args:
        model (torch.nn.Module): _description_
        train_dataloader (torch.utils.data.DataLoader): _description_
        test_dataloader (torch.utils.data.DataLoader): _description_
        optimizer (torch.optim.Optimizer): _description_
        loss_fn (torch.nn.Module): _description_
        epochs (int): _description_
        device (torch.device): _description_

    Returns:
        Dict[str, List]: _description_
    """   

    results = {'train_loss':[],
              'train_acc': [],
              'test_loss': [],
              'test_acc': []} 
    
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader= train_dataloader,
                                           loss_fn = loss_fn,
                                           optimizer=optimizer,
                                           device = device)
        
        test_loss, test_acc = test_step(model=model,
                                        dataloader = test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)
        
        print(
            f'Epoch: {epoch + 1} |'
            f'train_loss: {train_loss:.4f} |'
            f'train_acc: {train_acc:.4f} |'
            f'test_lossL {test_loss:.4f} | '
            f'test_acc: {test_acc:.4f}'
        )

        results['test_loss'].append(train_loss)
        results['train_acc'].append(train_acc)
        results['test_loss'].append(test_loss)
        results['test_acc'].append(test_acc)

        if writer:
            writer.add_scalars (main_tag = 'Accuracy',
                                tag_scalar_dict = {'train_acc': train_acc,
                                                'test_acc': test_acc},
                                                global_step = epoch)
            writer.add_scalars(main_tag = 'Loss',
                                tag_scalar_dict = {'train_loss':train_loss,
                                                 'test_acc':test_acc},
                                global_step=epoch)
            
            writer.add_graph (model= model,
                            input_to_model=torch.randn(32, 3, 224, 224).to(device))
        
    writer.close()

    return results


In [27]:
set_seeds()
results = train (model = model_B0,
                 train_dataloader=train_dataloader,
                 test_dataloader=test_dataloader,
                 optimizer=optimizer,
                 loss_fn=loss_fn,
                 epochs = 5,
                 device = device)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 |train_loss: 0.8159 |train_acc: 0.8398 |test_lossL 0.8180 | test_acc: 0.7955


 20%|██        | 1/5 [00:04<00:18,  4.58s/it]

Epoch: 2 |train_loss: 0.8094 |train_acc: 0.6719 |test_lossL 0.7170 | test_acc: 0.8258


 40%|████      | 2/5 [00:09<00:13,  4.60s/it]

...

 60%|██████    | 3/5 [00:13<00:09,  4.56s/it]

Epoch: 4 |train_loss: 0.6161 |train_acc: 0.7383 |test_lossL 0.6252 | test_acc: 0.8561


 80%|████████  | 4/5 [00:18<00:04,  4.54s/it]

Epoch: 5 |train_loss: 0.6619 |train_acc: 0.7422 |test_lossL 0.6354 | test_acc: 0.8258


100%|██████████| 5/5 [00:22<00:00,  4.54s/it]


## Dowlad dataset

In [29]:
data_10_percent_path = Path('../data/pizza_steak_sushi')
data_20_percent_path = Path('../data/pizza_steak_sushi_20_percent')
train_dir_10_percent = data_10_percent_path / 'train'
train_dir_20_percent = data_20_percent_path / 'train'
test_dir = data_10_percent_path / 'test'
print(f"Training directory 10%: {train_dir_10_percent}")
print(f"Training directory 20%: {train_dir_20_percent}")
print(f"Testing directory: {test_dir}")

Training directory 10%: ..\data\pizza_steak_sushi\train
Training directory 20%: ..\data\pizza_steak_sushi_20_percent\train
Testing directory: ..\data\pizza_steak_sushi\test


In [31]:
normalize = transforms.Normalize(mean = [0.485, 0.465, 0.406],
                                 std=[0.229, 0.224, 0.225])

simple_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

In [33]:
BATCH_SIZE =  32

train_dataloader_10_percent, test_dataloader, class_names =data_loaders.create_dataloaders(
    train_dir = train_dir_10_percent,
    test_dir = test_dir,
    transform = simple_transform,
    batch_size=BATCH_SIZE
) 

train_dataloader_20_percent, test_dataloader, class_names =data_loaders.create_dataloaders(
    train_dir = train_dir_20_percent,
    test_dir = test_dir,
    transform = simple_transform,
    batch_size=BATCH_SIZE
) 
# Find the number of samples/batches per dataloader (using the same test_dataloader for both experiments)
print(f"Number of batches of size {BATCH_SIZE} in 10 percent training data: {len(train_dataloader_10_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in 20 percent training data: {len(train_dataloader_20_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in testing data: {len(train_dataloader_10_percent)} (all experiments will use the same test set)")
print(f"Number of classes: {len(class_names)}, class names: {class_names}")

Number of batches of size 32 in 10 percent training data: 8
Number of batches of size 32 in 20 percent training data: 15
Number of batches of size 32 in testing data: 8 (all experiments will use the same test set)
Number of classes: 3, class names: ['pizza', 'steak', 'sushi']


In [38]:
effnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
effnetb2 = torchvision.models.efficientnet_b2(weights=effnetb2_weights)

print(summary (model=effnetb2,
         input_size = (32, 3, 224, 224),
         col_names = ['input_size', 'output_size', 'num_params', 'trainable'],
         col_width = 20,
         row_settings = ['var_names']
         ))

print (f'number of in_features ot final layer of EfficentB2 {len(effnetb2.classifier.state_dict()["1.weight"][0])}')
# print(f"Number of in_features to final layer of EfficientNetB2: {len(effnetb2.classifier.state_dict()['1.weight'][0])}")

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1408, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   64                   True
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 16, 112

In [55]:
OUT_FEATUREs = len (class_names)
def create_effnetb0():
    weignts = torchvision.models.EfficientNet_B0_Weights.DEFAULT
    model = torchvision.models.efficientnet_b0(weights=weights).to(device)

    for param in model.features.parameters():
        param.requires_grad=False

    set_seeds
    
    model.classifier=nn.Sequential(
        nn.Dropout(p = 0.2),
        nn.Linear(in_features=1280, out_features=OUT_FEATUREs)
    ).to(device)

    model.name = 'effnetb0'
    print (f'[INFO] Crated new {model.name} model')
    return model

def create_effnetb2():
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    model = torchvision.models.efficientnet_b2(weights=weights).to(device)

    for param in model.features.parameters():
        param.requires_grad=False
    
    set_seeds()

    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3),
        nn.Linear(in_features=1408, out_features=OUT_FEATUREs)
    ).to(device)

    model.name = 'effnetb2'
    print (f'[INFO] Creted new {model.name} model')

    return model


In [50]:
effnetb0 = create_effnetb0()

summary(model=effnetb0,
        input_size = (32, 3, 224, 224 ),
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width = 20,
        row_settings= ['var_names']
)


[INFO] Crated new effnetb0 model


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [57]:
effnetb2 = create_effnetb2()

summary(model=effnetb2,
        input_size = (32, 3, 224, 224 ),
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width = 20,
        row_settings= ['var_names']
)


[INFO] Creted new effnetb2 model


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1408, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

## Create Experiments and training code

In [58]:
num_epochs = [5, 10]
models = ['effnetb0', 'effnetb2']
train_dataloaders = {'data_10_percent': train_dataloader_10_percent,
                     'data_20_percent': train_dataloader_20_percent}

In [85]:
%%time
from going_modular.utils import save_model
set_seeds (seed=42)
experiment_number = 0

for dataloader_name, train_dataloader in train_dataloaders.items():
    for epochs in num_epochs:
        for model_name in models:
            experiment_number += 1
            print(f"[INFO] Experiment number: {experiment_number}")
            print(f"[INFO] Model: {model_name}")
            print(f"[INFO] DataLoader: {dataloader_name}")
            print(f"[INFO] Number of epochs: {epochs}")  

            if model_name == 'effnetb0':
                model = create_effnetb0()
            else:
                model = create_effnetb2()

            loss_fn = nn.CrossEntropyLoss()
            optimizer=torch.optim.Adam (params=model.parameters(), lr= 0.001)

            train(model= model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs = epochs,
                device = device,
                writer=create_writer (experiment_name = dataloader_name,
                                        model_name = model_name,
                                        extra = f'{epochs}_epochs'))

            save_filepath = f"07_{model_name}_{dataloader_name}_{epochs}_epochs.pth"
            save_model (model=model,
                target_dir = 'models',
                model_name = save_filepath)

            print ("_"*50 + "\n")
           

[INFO] Experiment number: 1
[INFO] Model: effnetb0
[INFO] DataLoader: data_10_percent
[INFO] Number of epochs: 5
[INFO] Crated new effnetb0 model
[INFO] Created SummaryWriter, saving to: runs\2024-09-17\data_10_percent\effnetb0\5_epochs..


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 |train_loss: 1.0303 |train_acc: 0.4375 |test_lossL 0.8738 | test_acc: 0.6629


 20%|██        | 1/5 [00:06<00:24,  6.04s/it]

Epoch: 2 |train_loss: 0.8515 |train_acc: 0.6406 |test_lossL 0.6976 | test_acc: 0.8561


...

Epoch: 9 |train_loss: 0.3426 |train_acc: 0.8875 |test_lossL 0.4337 | test_acc: 0.8674


 90%|█████████ | 9/10 [01:33<00:11, 11.05s/it]

Epoch: 10 |train_loss: 0.3785 |train_acc: 0.8604 |test_lossL 0.3763 | test_acc: 0.9489


100%|██████████| 10/10 [01:45<00:00, 10.58s/it]


[INFO] Saving model to: models\07_effnetb2_data_20_percent_10_epochs.pth
__________________________________________________

CPU times: total: 21min 7s
Wall time: 8min 52s


In [81]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 87220), started 0:19:46 ago. (Use '!kill 87220' to kill it.)

In [83]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 87220), started 0:25:00 ago. (Use '!kill 87220' to kill it.)